# L2 Clause Pipeline — Executable Reference

This notebook wires the deterministic L2 pieces together: candidate extraction, normalization, and validation.
Adapter (LLM) is optional and must be opt-in.

In [ ]:
# Setup: ensure repo root on sys.path for imports
import os, sys, json
# notebook cwd is level2/notebooks — go two levels up to project root
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from level2 import clause_extractor, normalizer, validator
from level2.llm_adapter import adapter as llm_adapter

INTENTS = [
    'investigate',
    'execute',
    'summarize',
    'ops',
]

CLAUSE_FIELDS = ['entity','operation','metric','time_window','environment','condition','constraint','output_format']

ModuleNotFoundError: No module named 'level2'

In [ ]:
def l2_pipeline(utterance: str, intent: str, use_adapter: bool = False) -> dict:
    """Run the L2 deterministic pipeline over an utterance.

    Returns the L2 output schema (dict).
    """
    if intent not in INTENTS:
        raise ValueError(f'unknown intent: {intent}')

    adapter_result = {}
    if use_adapter:
        adapter_result = llm_adapter.extract_clauses(utterance) or {}

    clauses, detectors = clause_extractor.extract_candidates(utterance, use_adapter=use_adapter, adapter_result=adapter_result)
    normalized = normalizer.normalize_clauses(clauses)
    decision_state, details, ambiguity = validator.validate(normalized, intent)

    decision_trace = {
        'detectors_fired': detectors,
        'hard_rules_failed': details.get('hard_rules_failed', []),
        'soft_rules_passed': details.get('soft_rules_passed', []),
        'alternatives_eliminated': {},
    }

    out = {
        'utterance': utterance,
        'intent': intent,
        'clauses': normalized,
        'first_pass': 'success' if decision_state == 'accepted' else ('partial' if decision_state == 'needs_clarification' else 'failure'),
        'ambiguity_report': ambiguity,
        'decision_state': decision_state,
        'decision_trace': decision_trace,
        'metadata': {
            'timestamp': __import__('datetime').datetime.utcnow().isoformat() + 'Z',
            'adapter_used': bool(use_adapter),
            'adapter_confidence': None,
        },
    }
    return out

In [ ]:
# Example runs
samples = [
    ("Restart web-01 in prod immediately", 'execute'),
    ("Show CPU and memory for web-01 for the last 5m", 'investigate'),
    ("Delete db-02 in prod", 'execute'),
]

for utt, intent in samples:
    result = l2_pipeline(utt, intent, use_adapter=False)
    print('---')
    print(utt)
    print(json.dumps(result, indent=2))

### Notes:
- This notebook is an executable reference. In production, the adapter should be implemented under `level2/llm_adapter` and adapter outputs must be re-validated by the deterministic validators.
- The functions here are intentionally conservative; they expose ambiguity rather than guessing.